In [ ]:
%matplotlib notebook

import numpy as np
import os
import os.path
import itertools
import json
import matplotlib.pyplot as plt
import h5py

from glob import glob

In [ ]:
set_name = 'test_11'
dir_name = 'abp_'+set_name

In [ ]:
cd /home/pierre/compute/ABP_lammps/probe_campaign_2

In [ ]:
pwd

In [ ]:

run_type = 'free'

parameter = 'tau_r'
v0 = 2

prefix = f'{dir_name}_v0_{int(100*v0):06d}_tau_r_*_{run_type}'

sets = glob(os.path.join(dir_name, prefix+'_*.json'))
sets.sort()

In [ ]:
prefix, sets

In [ ]:
cd ..

In [ ]:
all_msd = []
all_v0 = []
all_vsq = []
all_tau_r = []
all_vacf = []

for s in sets:
    glob_pattern = s[:-6]+'*_strided.h5'
    print(glob_pattern)
    runs = glob(glob_pattern)
    runs.sort()
    print(runs)
    with open(s, 'rb') as f:
        d = json.load(f)
    all_v0.append(d['v0'])
    all_tau_r.append(d['tau_r'])
    msd_data = []
    vsq_data = []
    vacf_data = []
    for r in runs:
        with h5py.File(r, 'r') as a:
            msd_data.append(a['msd'][:])
            r_dt = a['r_dt'][()]
            vsq_data.append(a['vacf'][0])
            vacf_data.append(a['vacf'][:])
 
    all_vsq.append(np.mean(vsq_data))
    all_msd.append(np.mean(msd_data, axis=0))
    all_vacf.append(np.mean(vacf_data, axis=0))


In [ ]:
n = len(all_msd[0])
t = np.arange(n)*r_dt

plt.figure()

all_D = []
for msd in all_msd:
    l, = plt.plot(t, msd)
    mask = (t>=100) * (t<=1000)
    slope, y0 = np.polyfit(t[mask], msd[mask], 1)
    plt.plot(t, y0 + slope*t, c=l.get_color(), ls='--')
    print(y0)
    all_D.append(slope/4)

In [ ]:
n = len(all_vacf[0])
t = np.arange(n)*v_dt

plt.figure()

for tau_r, vacf in zip(all_tau_r, all_vacf):
    plt.plot(t, vacf, label=str(tau_r))

plt.legend()

In [ ]:
all_D[0], T/gamma # damp = mass / gamma # gamma = mass / damp

In [ ]:
all_D

In [ ]:
plt.figure()
plt.plot(all_vsq, marker='o')

In [ ]:
D0 = 1/3

In [ ]:
plt.figure()

probe_mass = 0.3 #d['mass_probe']
sigma_probe = d['sigma_probe']

Pe_sq = np.array(all_vsq)*sigma_probe**2 / D0**2

plt.plot(all_tau_r, all_D, label='D')
plt.plot(all_tau_r, (1+2/3*Pe_sq)/360, label='1+2/3 Pe^2')
plt.plot(all_tau_r, probe_mass*np.array(all_vsq)/2, label='m v**2/2')

plt.legend()

In [ ]:
all_v0, all_D

In [ ]:
D_grav = all_D[-2]

In [ ]:
parameter = 'add_force'

run_type = 'add_force'

prefix = f'{dir_name}_v0_{int(100*v0):06d}_tau_r_*_{run_type}'

sets = glob(os.path.join(dir_name, prefix+'_*.json'))
sets.sort()

In [ ]:
sets

In [ ]:
all_x = []
all_f = []

for s in sets:
    runs = glob(s[:-6]+'*_processed.h5')
    with open(s, 'rb') as f:
        d = json.load(f)
    all_f.append(d['add_force'])
    x_data = []
    for r in runs:
        with h5py.File(r, 'r') as a:
            n = a['x'].shape[0]
            x_data.append(a['x'][:n//2])
            r_dt = a['r_dt'][()]
    all_x.append(np.mean(x_data, axis=0))



In [ ]:
plt.figure()

mu_data = []
t = np.arange(len(all_x[0]))*r_dt
for x, f in zip(all_x, all_f):
    l, = plt.plot(t, x)
    fit = np.polyfit(t, x, 1)
    plt.plot(t, np.poly1d(fit)(t), ls='--', color=l.get_color())
    if f > 0:
        mu_data.append(fit[0]/f)


In [ ]:
plt.figure()
plt.plot(mu_data)

In [ ]:
m, s = np.mean(mu_data, axis=0), np.std(mu_data, axis=0, ddof=1)

In [ ]:
s/m, s, m

In [ ]:
mu_grav = m

In [ ]:
set_name = 'test_08'
dir_name = 'abp_'+set_name

In [ ]:
parameter = 'gravity'

prefix = f'{dir_name}_{parameter}'

sets = glob(dir_name+'*_tau_r_001000_'+parameter+'*.json')
sets.sort()

In [ ]:
print(sets[0][:-7]+'*_strided.h5')

In [ ]:
all_y = []
all_g = []

for s in sets:
    runs = glob(s[:-7]+'_*_strided.h5')
    with open(s, 'rb') as f:
        d = json.load(f)
    if d['v0'] == 0.: continue
    all_g.append(d['gravity'])
    y_data = []
    for r in runs:
        with h5py.File(r, 'r') as a:
            y_data.append(a['y'][:])
            r_dt = a['r_dt'][()]
    all_y.append(np.concatenate(y_data))


In [ ]:
glob(sets[1][:-7]+'_*_strided.h5')

In [ ]:
slope_data = []

plt.figure()

i = 1
for y, g in zip(all_y, all_g):
    count, bins, patches = plt.hist(y, histtype='step', bins=30, density=True, label=str(i))
    i += 1
    mask = count > 1e-4
    bins = bins[:-1][mask]
    count = count[mask]
    mask = bins < 23
    bins = bins[mask]
    count = count[mask]
    print(g, sum(count))
    if True:
        slope, y0 = np.polyfit(bins, np.log(count), 1)
        plt.plot(bins, np.exp(y0+slope*bins), c=patches[0].get_edgecolor())
        slope_data.append(slope)

plt.yscale('log')
plt.legend()

In [ ]:
plt.figure()

plt.plot(slope_data[1:] / np.array(all_g)[1:], ls='', marker='o')

plt.plot(np.array(all_g)[1:]/slope_data[1:], ls='', marker='x')

In [ ]:
all_g, all_D[1:], all_g[::2]

In [ ]:
np.array(slope_data)[::2]

In [ ]:
mu = 0.225 # approx

force = 1

mu*force/all_D/2

In [ ]:
plt.figure()

# take delta = D / (mu force) -> balance of diffusive and sedimentation currents
# D = mu T -> delta = 1/gravity -> if 

plt.plot(1/np.array(slope_data)[::2], marker='o')
plt.plot(all_vsq[1:], marker='o')
#plt.plot(all_D[1:]/(mu*force), marker='o')



In [ ]:
DEBUG = {
    'free': True,
    'add_force': False,
    'gravity': False
}


TRACE = {
    'free': False,
    'add_force': False,
    'gravity': False
}


def process_free(prefix, p):
    runs = glob(prefix+'*_strided.h5')
    print(runs)
    D_data = []
    msd_data = []
    vsq_data = []
    l = None
    for r in runs:
        with h5py.File(r, 'r') as a:
            msd = a['msd'][:]
            msd_data.append(msd)
            n = len(msd)
            r_dt = a['r_dt'][()]
            vsq_data.append(a['vacf'][0])
        t = np.arange(n)*r_dt
        D_data.append(np.polyfit(t[n//32:n//16], msd[n//32:n//16], 1)[0]/4)
    if DEBUG['free']:
        m = np.mean(msd_data, axis=0)[::10]
        s = np.std(msd_data, axis=0, ddof=1)[::10]
        #plt.errorbar(t[::10], m-s, m+s, label=prefix, alpha=0.5)
        l, = plt.plot(t[::10], m, label=prefix)
        plt.plot(t[::10], m-s, color=l.get_color(), ls='--')
        plt.plot(t[::10], m+s, color=l.get_color(), ls='--')
        plt.plot(t[::10], np.mean(D_data)*4*t[::10], color=l.get_color(), ls=':')

    return np.array(D_data), np.array(vsq_data)


def process_add_force(prefix, p):
    runs = glob(prefix+'*_strided.h5')
    print(runs)
    f = p['add_force']
    x_data = []
    if TRACE['add_force']:
        plt.figure()
    for r in runs:
        with h5py.File(r, 'r') as a:
            n = a['x'].shape[0]
            x_data.append(a['x'][:n//2])
            r_dt = a['r_dt'][()]
        t = np.arange(len(x_data[-1]))*r_dt
        if TRACE['add_force']:
            plt.plot(t, x_data[-1])

    x_mean = np.mean(x_data, axis=0)
    fit = np.polyfit(t, x_mean, 1)
    if DEBUG['add_force']:
        l, = plt.plot(t, x_mean)
        plt.plot(t, np.poly1d(fit)(t), color=l.get_color())

    return fit[0]/f


def process_gravity(prefix, p):
    return 0

processors = {
    'free': process_free,
    'add_force': process_add_force,
    'gravity': process_gravity
}

In [ ]:
set_name = 'test_11'
dir_name = 'abp_'+set_name

os.chdir(os.path.expanduser(f'~/compute/ABP_lammps/probe_campaign_2/{dir_name}'))

with open(dir_name+'.json', 'r') as f:
    set_d = json.load(f)

fixed_name = 'v0'
fixed_value = 2
vary_name = 'tau_r'

values = [p for p in set_d['plans'] if p['parameter'] == vary_name][0]['values']

res = {
    'free': [],
    'add_force': [],
    'gravity': []
}

if any(DEBUG):
    plt.figure()

for v in values:
    print('---------------')
    print('values loop')
    D = 0 # analyze free
    p = set_d['base_params'].copy()
    p.update(**{vary_name: v})

    for study in set_d['studies']:
        print('---------------')
        print('study loop')
        results = []
        study_name = study['name']
        study_values = study['values']
        for study_value in study_values:
            prefix = f'{dir_name}_'
            for PPP in ['v0', 'tau_r']:
                v = p[PPP]
                prefix += f'{PPP}_{int(100*v):06d}_'

            print('---------------')
            print('study values loop')
            print('---------------')
            if study_value is not None:
                prefix += f"{study_name}_{int(100*study_value):06d}_"
            else:
                prefix += f"{study_name}_"
            p.update(**{study_name: study_value})

            print(prefix)
            
            results.append(processors[study_name](prefix, p))
        res[study_name].append(results)
    
if DEBUG['free']:
    plt.legend()
    #plt.xlim(0, 10000)

In [ ]:
all_D = np.mean([D for D, vsq in map(lambda x: x[0], res['free'])], axis=1)

all_vsq = np.mean([vsq for D, vsq in map(lambda x: x[0], res['free'])], axis=1)


In [ ]:
all_D

In [ ]:
list(map(len, [D for D, vsq in map(lambda x: x[0], res['free'])]))

In [ ]:
plt.figure()

m = np.mean(res['add_force'], axis=1)
s = np.std(res['add_force'], axis=1, ddof=1)

l, = plt.plot(values, m, label='mu')

plt.figure()

plt.plot(values, m-s, color=l.get_color(), ls='--')
plt.plot(values, m+s, color=l.get_color(), ls='--')

m = np.mean([D for D, vsq in map(lambda x: x[0], res['free'])], axis=1)
s = np.std([D for D, vsq in map(lambda x: x[0], res['free'])], axis=1, ddof=1)

l, = plt.plot(values, m, label='D')

plt.plot(values, m-s, color=l.get_color(), ls='--')
plt.plot(values, m+s, color=l.get_color(), ls='--')

plt.plot(values, np.mean([vsq for D, vsq in map(lambda x: x[0], res['free'])], axis=1), label='vsq')

plt.plot(values, 6.66 + np.array(values)**2/100*7.66)

plt.legend()

In [ ]:
set_d['studies']

In [ ]:
# strided version

set_name = 'test_11'
dir_name = 'abp_'+set_name

#os.chdir(os.path.expanduser(f'~/SIMU/20200430/{dir_name}'))
os.chdir(os.path.expanduser(f'~/compute/ABP_lammps/probe_campaign_2/{dir_name}'))

with open(dir_name+'.json', 'r') as f:
    set_d = json.load(f)

fixed_name = 'v0'
fixed_value = 2
vary_name = 'tau_r'

values = [p for p in set_d['plans'] if p['parameter'] == vary_name][0]['values']
   
plt.figure()

ax1 = plt.subplot(211)
ax2 = plt.subplot(212, sharex=ax1)

for v in values:
    print('---------------')
    print('values loop')
    D = 0 # analyze free
    p = set_d['base_params'].copy()
    p.update(**{vary_name: v})

    study = set_d['studies'][0]
    study_name = study['name']
    study_values = study['values']
    for study_value in study_values[:1]:
        prefix = f'{dir_name}_'
        for PPP in ['v0', 'tau_r']:
            v = p[PPP]
            prefix += f'{PPP}_{int(100*v):06d}_'

        if study_value is not None:
            prefix += f"{study_name}_{int(100*study_value):06d}_"
        else:
            prefix += f"{study_name}_"
        p.update(**{study_name: study_value})

        print(prefix)
        vacf = []
        for r in glob(prefix+'*_strided.h5'):
            with h5py.File(r, 'r') as a:
                vacf.append(a['vacf'][:])
                v_dt = a['v_dt'][()]
        vacf = np.mean(vacf, axis=0)
        t = np.arange(len(vacf))*v_dt
        l, = ax1.plot(t, vacf, label=f"v0 {p['v0']}")
        if p['v0'] > 0:
            ax1.axvline(p['L']/p['v0'], color=l.get_color())
            
    study = set_d['studies'][2]
    study_name = study['name']
    study_values = study['values']
    for study_value in study_values[:1]:
        prefix = f'{dir_name}_'
        for PPP in ['v0', 'tau_r']:
            v = p[PPP]
            prefix += f'{PPP}_{int(100*v):06d}_'

        if study_value is not None:
            prefix += f"{study_name}_{int(100*study_value):06d}_"
        else:
            prefix += f"{study_name}_"
        p.update(**{study_name: study_value})

        print(prefix)
        vacf = []
        for r in glob(prefix+'*_strided.h5'):
            with h5py.File(r, 'r') as a:
                vacf.append(a['vacf'][:])
                v_dt = a['v_dt'][()]
        vacf = np.mean(vacf, axis=0)
        t = np.arange(len(vacf))*v_dt
        l, = ax2.plot(t, vacf, label=f"v0 {p['v0']}")
        if p['v0'] > 0:
            ax2.axvline(p['L']/p['v0'], color=l.get_color())


plt.xlabel('time')
plt.ylabel('probe VACF')
plt.xlim(-.1, 50)

p_type = 'lin'
if p_type=='lin':
    ax1.set_ylim(-0.2,2)
    ax2.set_ylim(-0.2,2)
else:
    ax1.set_ylim(1e-3,20)
    ax1.set_yscale('log')
    ax2.set_ylim(1e-3,20)
    ax2.set_yscale('log')

plt.legend()

plt.savefig(dir_name+'_vacf_'+p_type+'.pdf')

In [ ]:
set_d['studies']

In [ ]:
np.sqrt(all_vsq)